<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#HW-setup" data-toc-modified-id="HW-setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>HW setup</a></span></li><li><span><a href="#Configuration" data-toc-modified-id="Configuration-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Configuration</a></span></li><li><span><a href="#Imports-and-helpers-definition" data-toc-modified-id="Imports-and-helpers-definition-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Imports and helpers definition</a></span></li><li><span><a href="#Constants-to-help-define-the-programs" data-toc-modified-id="Constants-to-help-define-the-programs-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Constants to help define the programs</a></span></li><li><span><a href="#Connect-to-server-and-devices" data-toc-modified-id="Connect-to-server-and-devices-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Connect to server and devices</a></span></li><li><span><a href="#Enable-PQSC-reference-clock-output-(not-really-needded)-enable-external-clock" data-toc-modified-id="Enable-PQSC-reference-clock-output-(not-really-needded)-enable-external-clock-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Enable PQSC reference clock output (not really needded) enable external clock</a></span></li><li><span><a href="#Establish-PQSC-->-HDAWG-ZSync-link" data-toc-modified-id="Establish-PQSC-->-HDAWG-ZSync-link-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Establish PQSC -&gt; HDAWG ZSync link</a></span></li><li><span><a href="#Configure-triggering" data-toc-modified-id="Configure-triggering-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Configure triggering</a></span></li><li><span><a href="#Create-HDAWG-waveforms" data-toc-modified-id="Create-HDAWG-waveforms-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Create HDAWG waveforms</a></span></li><li><span><a href="#Create-HDAWG-AWG-program-and-configure" data-toc-modified-id="Create-HDAWG-AWG-program-and-configure-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Create HDAWG AWG program and configure</a></span></li><li><span><a href="#Create-UHFQA-waveforms" data-toc-modified-id="Create-UHFQA-waveforms-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Create UHFQA waveforms</a></span></li><li><span><a href="#Create-UHFQA-AWG-program-and-configure" data-toc-modified-id="Create-UHFQA-AWG-program-and-configure-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Create UHFQA AWG program and configure</a></span></li><li><span><a href="#Configure-watching-device-(open-scope-in-UI)" data-toc-modified-id="Configure-watching-device-(open-scope-in-UI)-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>Configure watching device (open scope in UI)</a></span></li><li><span><a href="#Start-sequence" data-toc-modified-id="Start-sequence-14"><span class="toc-item-num">14&nbsp;&nbsp;</span>Start sequence</a></span></li><li><span><a href="#Plot-results" data-toc-modified-id="Plot-results-15"><span class="toc-item-num">15&nbsp;&nbsp;</span>Plot results</a></span></li><li><span><a href="#Example-scope-output" data-toc-modified-id="Example-scope-output-16"><span class="toc-item-num">16&nbsp;&nbsp;</span>Example scope output</a></span></li></ul></div>

# HW setup
```
    +------------------------(ExtClk 10MHz)----------------------+
    |                                                            |
    v                                                            v
[devPQCS] ------(ZSync)------> [devHDAWG] ------(DIO)------> [devUHFQA]
                                  (sigout0)                     (sigout0)
                                      |                             |
                                      +-----------+    +------------+
                                                  |    |
                                            (sigin0) (sigin1)
                                                  v    v
                                                [devWatch]
```
Devices are expected to be in power-on state at the beginning of this script. Transition from other states was not verified!

Tested with bitstream/FW revisions:
* PQCS: 66357/66351
* HDAWG: 66289/66326
* UHFQA: 66429/66429

# Configuration

In [1]:
def select_neuromancer():
    return 'dev10008', 'dev8216', 'dev2043', 'dev2029'

def select_wintermute():
    return 'dev10011', 'dev8047', 'dev2119', 'dev2058'

def select_countzero():
    return 'dev10007', 'dev8021', 'dev2027', 'dev2003'

def select_instruments(setup):
    SETUPS = {
        'neuromancer': select_neuromancer,
        'wintermute': select_wintermute,
        'countzero': select_countzero
    }
    # Get the function from switcher dictionary
    func = SETUPS.get(setup, lambda: "Invalid setup selected")
    # Execute the function
    return func()

In [2]:
setup = 'neuromancer'
devPQSC, devHDAWG, devUHFQA, devWatch = select_instruments(setup)
ZSyncPort = 0

dataServerIP = 'zicrunch1'
#dataServerIP = '10.42.0.238'
dataServerPort = 8004

# Number of sweep points
N_sweep = 5
# Number of averages per sweep point
N_averages = 1
# Number of echo pulses
N_echo = 3
# Time step increment for distance between R90 and Y180 pulse of each sweep point. 
# Note that the increment between the echo pulses (Y180) is twice the distance 
# between the first R90 and Y180 pulse and the last Y180 and R90 pulse.
t_delta = 40.0e-9
# Starting time
t_start = 40.0e-9

In [1]:
t_start

NameError: name 't_start' is not defined

# Imports and helpers definition

In [3]:
import time
import numpy as np
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
import zhinst.ziPython as zi

def awg_compile_upload_elf(awgModule, awgIndex, awg_program):
    """Compile and upload awg_program as .elf file"""
    awgModule.set('index', awgIndex)
    awgModule.set('compiler/sourcestring', awg_program)

    while awgModule.getInt('compiler/status') == -1:
        time.sleep(0.1)
    if awgModule.getInt('compiler/status') == 1:
        # compilation failed, raise an exception
        raise Exception(awgModule.getString('compiler/statusstring'))

    if awgModule.getInt('compiler/status') == 0:
        print("Compilation successful with no warnings, will upload the program to the instrument.")
    if awgModule.getInt('compiler/status') == 2:
        print("Compilation successful with warnings, will upload the program to the instrument.")
        print("Compiler warning: ", awgModule.getString('compiler/statusstring'))

    # Wait for the waveform upload to finish
    time.sleep(0.2)
    i = 0
    while (awgModule.getDouble('progress') < 1.0) and (awgModule.getInt('elf/status') != 1):
        p = awgModule.getDouble('progress')
        print(f'{i} awgModule progress: {p:.2f}')
        time.sleep(0.5)
        i += 1

    p = awgModule.getDouble('progress')
    print(f'{i} awgModule progress: {p:.2f}')
    if awgModule.getInt('elf/status') == 0:
        print("Upload to the instrument successful.")
    if awgModule.getInt('elf/status') == 1:
        raise Exception("Upload to the instrument failed.")
    time.sleep(0.5)
    
def awg_directory(daq):
    awg = daq.awgModule()
    awg.execute()
    return awg.get('awgModule/directory')['directory'][0]
        
def upload_awg_program(daq, device, index, program):
    awg = daq.awgModule()
    awg.set('device', device)
    awg.set('index', index)
    awg.execute()
    awg_compile_upload_elf(awg, index, program)
    daq.setInt(f'/{device}/awgs/0/single', 1)
    return awg

def time2time(t):
    '''Round a time value in seconds into a time value that is compatible with
    the common time of a UHFQA and an HDAWG instrument.'''
    return np.round(t*75e6)/75e6

def time2samples(t, fs, align=8):
    '''Convert a time value into a number of samples and ensure it is rounded
    to a number of samples divisible by the align argument.'''
    return int(align*(np.round((t*fs)/align)))

def mixer_predistortion_matrix(alpha, phi):
    '''
    predistortion matrix correcting for a mixer with amplitude
    mismatch "alpha" and skewness "phi" in degrees
 
    M = [ 1             tan(phi) ]
        [ 0   1/alpha * sec(phi)]
    '''
    predistortion_matrix = np.array(
        [[1,  np.tan(phi*2*np.pi/360)],
         [0, 1/alpha * 1/np.cos(phi*2*np.pi/360)]])
    return predistortion_matrix

def rotate_wave(wave_I, wave_Q, phase: float, unit: str = 'deg'):
    """
    Rotate a wave in the complex plane
        wave_I (array) : real component
        wave_Q (array) : imaginary component
        phase (float)  : desired rotation angle
        unit     (str) : either "deg" or "rad"
    returns:
        (rot_I, rot_Q) : arrays containing the rotated waves
    """
    if unit == 'deg':
        angle = np.deg2rad(phase)
    elif unit == 'rad':
        angle = angle
    else:
        raise ValueError('unit must be either "deg" or "rad"')
    rot_I = np.cos(angle)*wave_I - np.sin(angle)*wave_Q
    rot_Q = np.sin(angle)*wave_I + np.cos(angle)*wave_Q
    return rot_I, rot_Q

def save_complex_csv(directory, filename, wave_I, wave_Q, fs):
    n_wave = np.min((len(wave_I), len(wave_Q)))
    print(f'Saving {n_wave} ({1.0e9*n_wave/fs:.2f} ns) point complex waveform to {filename}.csv')
    np.savetxt(f'{directory}/awg/waves/{filename}.csv',            
               np.stack((wave_I, wave_Q), 1))

def save_csv(directory, filename, wave, fs):
    n_wave = len(wave)
    print(f'Saving {n_wave} ({1.0e9*n_wave/fs:.2f} ns) point waveform to {filename}.csv')
    np.savetxt(f'{directory}/awg/waves/{filename}.csv', wave)

def gauss_pulse(amp: float, sigma_length: float, nr_sigma: int=4,
                sampling_rate: float=2.4e9, axis: str='x', phase: float=0,
                phase_unit: str='deg',
                motzoi: float=0, delay: float=0,
                subtract_offset: str='average'):
    '''
    All inputs are in s and Hz.
    phases are in degree.

    Args:
        amp (float):
            Amplitude of the Gaussian envelope.
        sigma_length (float):
            Sigma of the Gaussian envelope.
        nr_sigma (int):
            After how many sigma the Gaussian is cut off.
        sampling_rate (float):
            Rate at which the pulse is sampled.
        axis (str):
            Rotation axis of the pulse. If this is 'y', a 90-degree phase is
            added to the pulse, otherwise this argument is ignored.
        phase (float):
            Phase of the pulse.
        phase_unit (str):
            Unit of the phase (can be either "deg" or "rad")
        motzoi (float):
            DRAG-pulse parameter.
        delay (float):
            Delay of the pulse in s.
        subtract_offset (str):
            Instruction on how to subtract the offset in order to avoid jumps
            in the waveform due to the cut-off.
            'average': subtract the average of the first and last point.
            'first': subtract the value of the waveform at the first sample.
            'last': subtract the value of the waveform at the last sample.
            'none', None: don't subtract any offset.

    Returns:
        pulse_I, pulse_Q: Two quadratures of the waveform.
    '''
    sigma = sigma_length  # old legacy naming, to be replaced
    length = sigma*nr_sigma

    t_step = 1/sampling_rate
    mu = length/2. - 0.5*t_step  # center should be offset by half a sample
    t = np.arange(0, nr_sigma*sigma, t_step)

    gauss_env = amp*np.exp(-(0.5 * ((t-mu)**2) / sigma**2))
    deriv_gauss_env = motzoi * -1 * (t-mu)/(sigma**1) * gauss_env

    # Subtract offsets
    if subtract_offset.lower() == 'none' or subtract_offset is None:
        # Do not subtract offset
        pass
    elif subtract_offset.lower() == 'average':
        gauss_env -= (gauss_env[0]+gauss_env[-1])/2.
        deriv_gauss_env -= (deriv_gauss_env[0]+deriv_gauss_env[-1])/2.
    elif subtract_offset.lower() == 'first':
        gauss_env -= gauss_env[0]
        deriv_gauss_env -= deriv_gauss_env[0]
    elif subtract_offset.lower() == 'last':
        gauss_env -= gauss_env[-1]
        deriv_gauss_env -= deriv_gauss_env[-1]
    else:
        raise ValueError('Unknown value "{}" for keyword argument '
                         '"subtract_offset".'.format(subtract_offset))

    delay_samples = delay*sampling_rate

    # generate pulses
    Zeros = np.zeros(int(delay_samples))
    G = np.array(list(Zeros)+list(gauss_env))
    D = np.array(list(Zeros)+list(deriv_gauss_env))

    if axis == 'y':
        phase += 90

    pulse_I, pulse_Q = rotate_wave(G, D, phase=phase, unit=phase_unit)

    return pulse_I, pulse_Q

# Constants to help define the programs

In [4]:
# Constants defining hd and UHFQA signals
fs_uhfqa = 1.8e9
fs_hdawg = 2.4e9

# Offset to start of hd waveforms
t_offset_start = time2time(30e-9)

# Section 0
t_qubit_rx180 = time2time(100e-9)
sigma_qubit_rx180 = 20e-9

# Section 1
t_qubit_ry90 = time2time(50e-9)
sigma_qubit_ry90 = 20e-9

# Section 2
t_qubit_readout = time2time(300e-9)
t_qubit_integration = time2time(200e-9)
t_offset_qubit_integration = time2time(100e-9)

# Section 3
t_wait_100ns = time2time(100e-9)

# IF of qubit readout
f_uhfqa_readout = 100e6

# IF of qubit excitation
f_qubit_excitation = 50e6

# Readout mixer parameters
ro_mixer_alpha = 0.9
ro_mixer_phi = 2
ro_mixer_offset_I = 0.02
ro_mixer_offset_Q = -0.05

# Qubit control mixer parameters
q0_mixer_alpha = 1.1
q0_mixer_phi = -3
q0_mixer_offset_I = -0.03
q0_mixer_offset_Q = -0.06

# Connect to server and devices

In [5]:
# Conenct to data server        
daq = zi.ziDAQServer(dataServerIP, dataServerPort, 6)
#daq.disconnectDevice(f'{devPQSC}')
#daq.disconnectDevice(f'{devHDAWG}')
#daq.disconnectDevice(f'{devUHFQA}')
#daq.disconnectDevice(f'{devWatch}')
daq.connectDevice(f'{devPQSC}',  '1gbe')
daq.connectDevice(f'{devHDAWG}', '1gbe')
daq.connectDevice(f'{devUHFQA}', '1gbe')
daq.connectDevice(f'{devWatch}', '1gbe')

# Enable PQSC reference clock output (not really needded) enable external clock

In [6]:
# ------------------------------------- PQSC
## Reference clock output
# Enable
daq.setInt(f'/{devPQSC}/system/clocks/referenceclock/out/enable', 1)
# 10MHz
daq.setInt(f'/{devPQSC}/system/clocks/referenceclock/out/freq', 10)
# External clock
daq.setInt(f'/{devPQSC}/system/clocks/referenceclock/in/source', 1)

# ------------------------------------- UHFQA
## Clock source
# External 10MHz (from PQSC)
daq.setInt(f'/{devUHFQA}/system/extclk', 1)

# Establish PQSC -> HDAWG ZSync link

In [ ]:
# Wait a little to make sure the previous commands are done
time.sleep(5)

# ------------------------------------- HDAWG
## Reference Clock
# ZSync
daq.setInt(f'/{devHDAWG}/system/clocks/referenceclock/source', 2)

# ------------------------------------- PQSC
# Reset port
daq.setInt(f'/{devPQSC}/zsyncs/{ZSyncPort}/connection/reset', 1)
time.sleep(2) # Wait for link to be established
# TODO:
# If PQSC is started too early, trigger may not work.
# Elaborate on condition for established ZSync link.

# Configure triggering
PQSC  ===(ZSync)==>  HDAWG  ===(ZSync-pass-through-DIO)==>  UHFQA

In [ ]:
# ------------------------------------- PQSC
# Will transmit 0xFFFFFFFF by default over ZSync with every trigger event - no special config

# ------------------------------------- HDAWG
## DIO
daq.setInt(f'/{devHDAWG}/raw/dios/0/mode', 0x042)

# ------------------------------------- HDAWG
## DIO
if daq.getInt(f'/{devHDAWG}/system/fpgarevision') <= 66289:
    # DIO mux (old style configuration):
    # 1st nibble (DIO-out): 2 = ZSync-in to DIO-out (1 = setDIO to DIO-out, 0 = user-val to DIO-out)
    # 2nd nibble (AWG-DIO-in): 4 = ZSync-in to waitDIOTrigger (2 = DIO-in to waitDIOTrigger, 1 = "2"+"4")
    # 3rd nibble (ZSync-out): 0 = ? (1 = DIO-in to ZSync-out, 2 = setDIO to ZSync-out)
    daq.setInt(f'/{devHDAWG}/raw/dios/0/mode', 0x042)
else:
    # DIO mux configuration (new style configuration)
    # 0 => Default mode - setDIO to DIO-out, DIO-in to waitDIOTrigger
    # 1 => QCCS mode - ZSync-in to DIO-out, ZSync-in to waitDIOTrigger
    daq.setInt(f'/{devHDAWG}/raw/dios/0/mode', 0x001)

# DIO FPGA: 1 => 50 MHz resampling of the DIO data.
daq.setInt(f'/{devHDAWG}/raw/dios/0/extclk', 0x1)
# DIO drive on bits 16-31, receive on bits 0-15 (DIOLink protocol)
daq.setInt(f'/{devHDAWG}/dios/0/drive', 0xc)
# waitDIOTrigger should _not_ work in obsolete strobe mode
daq.setInt(f'/{devHDAWG}/awgs/0/dio/strobe/slope', 0)
# Below settings are currently needed for HDAWG triggering from PQSC over ZSync. May change!
daq.setInt(f'/{devHDAWG}/awgs/0/dio/valid/polarity', 2)
daq.setInt(f'/{devHDAWG}/awgs/0/dio/valid/index', 0)

# ------------------------------------- UHFQA
# Set the mode to output Qubit results
daq.setInt(f'/{devUHFQA}/dios/0/mode', 0x2)
# DIO FPGA: 2 => 50 MHz resampling of the DIO data (undocumented).
daq.setInt(f'/{devUHFQA}/dios/0/extclk', 0x2)
# DIO drive on bits 0-15, receive on bits 16-31 (DIOLink protocol)
daq.setInt(f'/{devUHFQA}/dios/0/drive', 0x3)
# waitDIOTrigger should _not_ work in obsolete strobe mode
daq.setInt(f'/{devUHFQA}/awgs/0/dio/strobe/slope', 0)
# "high" state of DIO bit 16 indicates valid trigger event for waitDIOTrigger
daq.setInt(f'/{devUHFQA}/awgs/0/dio/valid/polarity', 2)
daq.setInt(f'/{devUHFQA}/awgs/0/dio/valid/index', 16)

# Create HDAWG waveforms

In [ ]:
n_offset_start_hd = time2samples(t_offset_start, fs_hdawg, 16)
w_offset_start_hd = np.zeros(n_offset_start_hd)
save_csv(awg_directory(daq), 'signal_hdawg_wait_000000', w_offset_start_hd, fs_hdawg)

n_qubit_rx180_hd = time2samples(t_qubit_rx180, fs_hdawg, 16)
w_qubit_rx180_I_hd, w_qubit_rx180_Q_hd = \
  gauss_pulse(1.0, sigma_qubit_rx180, nr_sigma=t_qubit_rx180/sigma_qubit_rx180, sampling_rate=2.4e9)
save_csv(awg_directory(daq), 'signal_I_hdawg_qubit_excitation_000000', w_qubit_rx180_I_hd, fs_hdawg)
save_csv(awg_directory(daq), 'signal_Q_hdawg_qubit_excitation_000000', w_qubit_rx180_Q_hd, fs_hdawg)

n_qubit_ry90_hd = time2samples(t_qubit_ry90, fs_hdawg, 16)
w_qubit_ry90_I_hd, w_qubit_ry90_Q_hd = \
  gauss_pulse(1.0, sigma_qubit_ry90, nr_sigma=t_qubit_ry90/sigma_qubit_ry90, sampling_rate=2.4e9, axis='y')
save_csv(awg_directory(daq), 'signal_I_hdawg_qubit_excitation_000001', w_qubit_ry90_I_hd, fs_hdawg)
save_csv(awg_directory(daq), 'signal_Q_hdawg_qubit_excitation_000001', w_qubit_ry90_Q_hd, fs_hdawg)

n_qubit_readout_hd = time2samples(t_qubit_readout, fs_hdawg, 16)
w_qubit_readout_hd = np.zeros(n_qubit_readout_hd)
save_csv(awg_directory(daq), 'signal_hdawg_qubit_readout_000000', w_qubit_readout_hd, fs_hdawg)

n_wait_100ns_hd = time2samples(t_wait_100ns, fs_hdawg, 16)
w_wait_100ns_hd = np.zeros(n_wait_100ns_hd)
save_csv(awg_directory(daq), 'signal_hdawg_wait_100ns_000000', w_wait_100ns_hd, fs_hdawg)

# Create HDAWG AWG program and configure

In [ ]:
# HDAWG program
awgprg_hd = '''
// Standard trigger
waitDIOTrigger();
resetOscPhase();
'''

awgprg_hd += '''
// Alignment with UHFQA
playWave("signal_hdawg_wait_000000");
'''

for i in range(N_sweep):
    awgprg_hd += f'''
// Sweep point {i+1} of {N_sweep}

// Section 0
playWave(1, 2, "signal_I_hdawg_qubit_excitation_000000", 1, 2, "signal_Q_hdawg_qubit_excitation_000000");'''
    for j in range(N_echo):
        awgprg_hd += f'''
playZero({time2samples(time2time(t_start + i*t_delta), 2.4e9)});'''
        awgprg_hd += f'''
playWave(1, 2, "signal_I_hdawg_qubit_excitation_000001", 1, 2, "signal_Q_hdawg_qubit_excitation_000001");'''
        awgprg_hd += f'''
playZero({time2samples(time2time(t_start + i*t_delta), 2.4e9)});'''

    awgprg_hd += f'''
playWave(1, 2, "signal_I_hdawg_qubit_excitation_000000", 1, 2, "signal_Q_hdawg_qubit_excitation_000000");

// Section 1
playWave("signal_hdawg_qubit_readout_000000");
    '''

    awgprg_hd += '''
// Section 2
playWave("signal_hdawg_wait_100ns_000000");
    '''
    
print(awgprg_hd)

# No strict need to set oscillatorcontrol to 0 first; nodes are still accessible
daq.setInt(f'/{devHDAWG}/system/awg/oscillatorcontrol', 1)
daq.setDouble(f'/{devHDAWG}/oscs/0/freq', f_qubit_excitation)
awg_hd = upload_awg_program(daq, devHDAWG, 0, awgprg_hd)

## Output
# Enable
daq.setInt(f'/{devHDAWG}/sigouts/0/on', 1)
daq.setInt(f'/{devHDAWG}/sigouts/1/on', 1)
# Modulation on (use symmetric selection for output 1 and 2)
daq.setInt(f'/{devHDAWG}/awgs/0/outputs/0/modulation/mode', 1)
daq.setInt(f'/{devHDAWG}/awgs/0/outputs/1/modulation/mode', 2)
# Single-sideband modulation
daq.setDouble(f'/{devHDAWG}/sines/1/oscselect', 0)
daq.setDouble(f'/{devHDAWG}/sines/0/phaseshift', 90)
# Use matrix multiplication in hardware
M = mixer_predistortion_matrix(q0_mixer_alpha, q0_mixer_phi)
for i in range(2):
    for j in range(2):
        daq.setDouble(f'/{devHDAWG}/awgs/0/outputs/{i}/gains/{j}', M[i][j])
# Mixer offset compensation
daq.setDouble(f'/{devHDAWG}/sigouts/0/offset', q0_mixer_offset_I)
daq.setDouble(f'/{devHDAWG}/sigouts/1/offset', q0_mixer_offset_Q)

# Create UHFQA waveforms

In [ ]:
M = mixer_predistortion_matrix(ro_mixer_alpha, ro_mixer_phi)

n_qubit_rx180_uhf = time2samples(t_qubit_rx180, fs_uhfqa, 8)
w_qubit_rx180_uhf = np.zeros(n_qubit_rx180_uhf)
save_csv(awg_directory(daq), 'signal_uhfqa_qubit_excitation_000000', w_qubit_rx180_uhf, fs_uhfqa)

n_qubit_ry90_uhf = time2samples(t_qubit_ry90, fs_uhfqa, 8)
w_qubit_ry90_uhf = np.zeros(n_qubit_ry90_uhf)
save_csv(awg_directory(daq), 'signal_uhfqa_qubit_excitation_000001', w_qubit_ry90_uhf, fs_uhfqa)

n_qubit_readout_uhf = time2samples(t_qubit_readout, fs_uhfqa, 8)
w_qubit_readout_I_uhf = np.cos(2*np.pi*f_uhfqa_readout*np.arange(n_qubit_readout_uhf)/fs_uhfqa)
w_qubit_readout_Q_uhf = np.sin(2*np.pi*f_uhfqa_readout*np.arange(n_qubit_readout_uhf)/fs_uhfqa)
w_qubit_readout = np.dot(M, np.vstack((w_qubit_readout_I_uhf, w_qubit_readout_Q_uhf)))
save_complex_csv(awg_directory(daq), 'signal_uhfqa_qubit_readout_000000', 
                 w_qubit_readout[0], w_qubit_readout[1], fs_uhfqa)

n_wait_100ns_uhf = time2samples(t_wait_100ns, fs_uhfqa, 8)
w_wait_100ns_uhf = np.zeros(n_wait_100ns_uhf)
save_csv(awg_directory(daq), 'signal_uhfqa_wait_100ns_000000', w_wait_100ns_uhf, fs_uhfqa)

# Create UHFQA AWG program and configure

In [ ]:
# ------------------------------------- UHFQA
## AWG
# UHFQA program
awgprg_uhf = '''
// Standard trigger
waitDIOTrigger();
'''

for i in range(N_sweep):
    awgprg_uhf += f'''
// Sweep point {i+1} of {N_sweep}

// Section 0
playWave("signal_uhfqa_qubit_excitation_000000");
playZero({time2samples(time2time(t_start + i*t_delta), 1.8e9)});'''
    for j in range(N_echo):
        if j > 0:
            awgprg_uhf += f'''
playZero({time2samples(time2time(2*(t_start + i*t_delta)), 1.8e9)});'''
        awgprg_uhf += f'''
playWave("signal_uhfqa_qubit_excitation_000001");'''

    awgprg_uhf += f'''
playZero({time2samples(time2time(t_start + i*t_delta), 1.8e9)});
playWave("signal_uhfqa_qubit_excitation_000000");
'''

    awgprg_uhf += '''
// Section 1
playWave("signal_uhfqa_qubit_readout_000000");
startQAResult();
'''

    awgprg_uhf += '''
// Section 2
playWave("signal_uhfqa_wait_100ns_000000");
'''

print(awgprg_uhf)
awg_uhf = upload_awg_program(daq, devUHFQA, 0, awgprg_uhf)

## Output
# Enable
daq.setInt(f'/{devUHFQA}/sigouts/0/on', 1)
daq.setInt(f'/{devUHFQA}/sigouts/1/on', 1)
# Mixer offset compensation
daq.setDouble(f'/{devUHFQA}/sigouts/0/offset', ro_mixer_offset_I)
daq.setDouble(f'/{devUHFQA}/sigouts/1/offset', ro_mixer_offset_Q)

# Modulation off
daq.setInt(f'/{devUHFQA}/awgs/0/outputs/0/mode', 0)
daq.setInt(f'/{devUHFQA}/awgs/0/outputs/1/mode', 0)

# Weight function
n_qubit_integration_uhf = time2samples(t_qubit_integration, 1.8e9, 4)
w_real = np.cos(2*np.pi*f_uhfqa_readout*np.arange(n_qubit_integration_uhf)/fs_uhfqa)
w_imag = np.sin(2*np.pi*f_uhfqa_readout*np.arange(n_qubit_integration_uhf)/fs_uhfqa)

print(f'Saving {n_qubit_integration_uhf} point waveform for weighting vectors')
np.savetxt(f'{awg_directory(daq)}/awg/waves/sweep_uhfqa_000000.csv', w_real, delimiter=',')
np.savetxt(f'{awg_directory(daq)}/awg/waves/sweep_uhfqa_000001.csv', w_imag, delimiter=',')

for i in range(1):
    daq.setVector(f'/{devUHFQA}/qas/0/integration/weights/{i}/real', w_real)
    daq.setVector(f'/{devUHFQA}/qas/0/integration/weights/{i}/imag', w_imag)
    daq.setComplex(f'/{devUHFQA}/qas/0/rotations/{i}', 0.1 + 1j*0.0)
    
# Integration mode
daq.setInt(f'/{devUHFQA}/qas/0/integration/mode', 0)
daq.setDouble(f'/{devUHFQA}/qas/0/integration/length', time2samples(t_qubit_integration, 1.8e9, 4))
daq.setDouble(f'/{devUHFQA}/qas/0/delay', time2samples(t_offset_qubit_integration, 1.8e9, 4))

# Result acquisition
daq.setDouble(f'/{devUHFQA}/qas/0/result/length', N_sweep)
daq.setDouble(f'/{devUHFQA}/qas/0/result/averages', N_averages)

# Configure watching device (open scope in UI)

In [ ]:
# ------------------------------------- Watching / scope

## Scope
daq.setInt(f'/{devWatch}/scopes/0/length', 16000)
# Enable both channels
daq.setInt(f'/{devWatch}/scopes/0/channel', 3)
# Enable trigger (default: rising edge, sigin0)
daq.setInt(f'/{devWatch}/scopes/0/trigenable', 1)
# Trigger on input 2
daq.setInt(f'/{devWatch}/scopes/0/trigchannel', 1)
# Level 350mv
daq.setDouble(f'/{devWatch}/scopes/0/triglevel', 0.35)
# Reference point
daq.setDouble(f'/{devWatch}/scopes/0/trigreference', 0.1)
# Enable segmented mode
daq.setInt(f'/{devWatch}/scopes/0/segments/enable', 0)
# Record ten segments (to match sweep points)
daq.setInt(f'/{devWatch}/scopes/0/segments/count', 1)
# Start
daq.setInt(f'/{devWatch}/scopes/0/single', 1)
daq.setInt(f'/{devWatch}/scopes/0/enable', 1)

print(f'Open scope tab for "{devWatch}" in LabOne UI - \
observe gaussian modulated pulses on sigin0 and a square modulated pulse on sigin1 \
(ideally without jitter)')

# Start sequence

In [ ]:
# Clear any errors
daq.setDouble(f'/{devUHFQA}/qas/0/result/reset', 1)
              
# Monitor AWG status
daq.subscribe(f'/{devHDAWG}/awgs/0/sequencer/status')
daq.subscribe(f'/{devUHFQA}/awgs/0/sequencer/status')
daq.subscribe(f'/{devWatch}/scopes/0/wave')

# ------------------------------------- PQSC
## Trigger output
# Trigger every 2us
daq.setDouble(f'/{devPQSC}/execution/holdoff', 2e-06)
# 1G triggers (keep triggers generated long enough)
daq.setDouble(f'/{devPQSC}/execution/repetitions', 1)

scope_data = None

# Outer loop for averaging
for _ in range(N_averages):
    print(f'Average {_+1} of {N_averages}...')
    
    daq.setInt(f'/{devHDAWG}/awgs/0/enable', 1)
    daq.setInt(f'/{devUHFQA}/awgs/0/enable', 1)
    status = {
        f'/{devHDAWG}/awgs/0/sequencer/status': True,
        f'/{devUHFQA}/awgs/0/sequencer/status': True
    }
    daq.sync()

    # Start
    daq.setInt(f'/{devPQSC}/execution/enable', 1)

    # Wait for AWG's to finish
    done = False
    while not done and scope_data is None:
        data = daq.poll(0.01, 1000, 4, True)
        done = True
        for path in status:
            if path in data:
                status[path] = data[path]['value'][-1] != 0

            if status[path]:
                done = False

        if not done:
            print("    Waiting for AWG's to finish")
            
        path = f'/{devWatch}/scopes/0/wave'
        if path in data:
            scope_data = data[path]
            
        if scope_data is None:
            print("    Waiting for scope data")
    print('    Done')

daq.unsubscribe(f'/{devHDAWG}/awgs/0/sequencer/status')
daq.unsubscribe(f'/{devUHFQA}/awgs/0/sequencer/status')
daq.unsubscribe(f'/{devWatch}/scopes/0/wave')

# Plot results

In [ ]:
plt.figure()
#plt.subplot(311)
N = scope_data[0]['totalsamples']
dt = scope_data[0]['dt']
t0 = scope_data[0]['timestamp']
t1 = scope_data[0]['triggertimestamp']
t = 1.0e9*dt*(np.arange(N)-N+t0-t1)
scaling = scope_data[0]['channelscaling']
sigin0 = scaling[0]*np.reshape(scope_data[0]['wave'], 2*N)[0::2]
sigin1 = scaling[1]*np.reshape(scope_data[0]['wave'], 2*N)[1::2]
plt.title('Complete measurement')
plt.plot(t, sigin0, label='sigin0 (HDAWG)')
plt.plot(t, sigin1, label='sigin1 (UHFQA)')
plt.xlabel('Time (ns)')
plt.ylabel('Amplituded (V)')
plt.xlim((t[0], t[-1]))
plt.legend()
plt.grid()
plt.show()

# Example scope output

![alt text](scope_t2.png "Example Scope output")